# 데이터 전처리 모듈

## 1. 라이브러리 설치

In [ ]:
# konlpy, Mecab 설치 및 로드
# Reference: https://teddylee777.github.io/colab/colab-mecab
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

from konlpy.tag import Mecab
mecab = Mecab()

## 2. 전처리 함수 정의
 - (이 부분만 .py로 따로 빼고 나중에 import로 불러와 사용할 수도 있음)
 - 불용어 사전 생성순서 유의

In [2]:
# 불용어 사전 생성
def MakeStopWord(DF):
    # 1. 부사, 형용사적 의미를 갖는 명사 및 국가명
    stopword = ['정말', '진짜', '최고', '그냥', '마지막', '다시', '완전', '정도', '역시', '처음', '지금', '이건', '별로', '때문', '뭔가', '가장', '조금', '제일',
                '영화', '관람객', '생각', '평점', '하나', '작품', '아이', '최악', '일본', '미국', '중국', '호주', '프랑스', '한국', '인도']
    # 판타지 General word
    # stopword.extend(['연기', '스토리', '기대', '사람', '배우', '사랑', '내용', '재미'])
    # 드라마 General word
    # stopword.extend(['연기', '배우', '사람', '감독', '스토리', '내용', '이야기'])
    
    # 2. 영화 타이틀
    for title in DF['movie_name'].unique().tolist():
        stopword.append(title)

    return stopword


# 명사추출 함수
def NNGNNP(sentence):
    nouns = []
    result = mecab.pos(sentence)
    for element in result:
        if element[1] == 'NNG' or element[1] == 'NNP':
            nouns.append(element[0])
    return nouns


# 불용어 제거 함수
def RemoveStopWord(ls):
    clean_review = []
    for element in ls:
        if len(element) == 1 or element in stopword:
            continue
        clean_review.append(element)
    return clean_review


# 통합 전처리 함수 (입력: DataFrame)
def Preprocessing(DF):
    # 결측치, 중복값 제거
    DF = DF.dropna()
    DF = DF.drop_duplicates(['review'])

    # 'Review'라는 새로운 column 생성 후, review에서 명사추출 및 불용어 제거한 값 할당
    DF['Review'] = ''
    DF['Review'] = DF['review'].apply(NNGNNP)
    DF['Review'] = DF['Review'].apply(RemoveStopWord)

    # 전처리 후 길이가 1이하 리뷰 제거
    review_len = 2
    DF = DF[DF['Review'].apply(len) >= review_len]
    
    # 감성 Labeling (평점 8점 이상 긍정, 3점 이하 부정)
    positive_score = 8
    negative_score = 3
    DF['sentiment'] = np.where(DF.score >= positive_score, 1,
                            np.where(DF.score <= negative_score, 0, 100))

    # index 초기화
    DF = DF.reset_index()
    DF = DF.drop(['index'], axis=1)

    return DF

## 3. 데이터 로드
 - ``영화 장르(genre)``와 ``파일 경로(PATH)`` 변경해서 사용

In [31]:
import pandas as pd
import numpy as np
import pickle

# 파일 경로(PATH)와 파일명(file_name) 변경해서 사용
genre = '판타지'
PATH = "/content/drive/MyDrive/final-project/data"
file_name = f"/movie_crawling_{genre}.csv"

df = pd.read_csv(PATH + file_name, index_col=0)

# 불러온 데이터 확인
df.head()

,genre,movie_name,score,review
0,판타지,반지의 제왕: 왕의 귀환,10,내가 극장에서 이 영화를 봣다는게 너무 자랑스럽고 감동적입니다
1,판타지,반지의 제왕: 왕의 귀환,10,현존하는 최고의 판타지영화... 퀄리티가 절대 애들용이 아님
2,판타지,반지의 제왕: 왕의 귀환,10,왜 이영화를이제서야봤는지ㅠㅠ 일편부터연달아보는데 재밌다가도 시간가는게너무안타까웠음ㅠ...
3,판타지,반지의 제왕: 왕의 귀환,10,몇년이 지난 영화인데도 소름돋는영화.
4,판타지,반지의 제왕: 왕의 귀환,10,반지의제왕 안본사람은 있어도한번본 사람은 없다는게 사실인가요


## 4. 크롤링 데이터 전처리

In [32]:
stopword = MakeStopWord(df)
df = Preprocessing(df)

df.head(10)

,genre,movie_name,score,review,Review,sentiment
0,판타지,반지의 제왕: 왕의 귀환,10,내가 극장에서 이 영화를 봣다는게 너무 자랑스럽고 감동적입니다,"[극장, 자랑, 감동]",1
1,판타지,반지의 제왕: 왕의 귀환,10,현존하는 최고의 판타지영화... 퀄리티가 절대 애들용이 아님,"[현존, 판타지, 퀄리티]",1
2,판타지,반지의 제왕: 왕의 귀환,10,왜 이영화를이제서야봤는지ㅠㅠ 일편부터연달아보는데 재밌다가도 시간가는게너무안타까웠음ㅠ...,"[시간, 해리포터, 반지, 제왕, 해리포터, 명작, 가슴, 며칠, 행복]",1
3,판타지,반지의 제왕: 왕의 귀환,10,반지의제왕 안본사람은 있어도한번본 사람은 없다는게 사실인가요,"[반지, 제왕, 사람, 사람, 사실]",1
4,판타지,반지의 제왕: 왕의 귀환,10,ㅅㅂ 이거 안보고 판타지 논하지 마라. 이거 뛰어넘을 판타지 절대 없을거다. 해리포...,"[판타지, 판타지, 해리포터, 장난]",1
5,판타지,반지의 제왕: 왕의 귀환,10,한때 반지시리즈와 해리포터를 놓고 판타지영화계의 양대산맥 어쩌고 비교하는 멍청이들이...,"[한때, 반지, 시리즈, 해리포터, 판타지, 산맥, 비교, 멍청이, 반지, 역사, ...",1
6,판타지,반지의 제왕: 왕의 귀환,10,솔직히 아바타는 이영화에 비해선 거품이다,"[아바타, 거품]",1
7,판타지,반지의 제왕: 왕의 귀환,10,"이건 영화라는 장르를 넘어섰다. 100년후에도 영화라는 문화를 다룰때, 언급될 작품...","[장르, 문화, 언급]",1
8,판타지,반지의 제왕: 왕의 귀환,10,톨킨이 이 영화를 본다면 어떤 반응을 할 지 궁금하다.. 영화화는 절대 불가능하다는...,"[톨킨, 반응, 가능, 원작자, 원작, 왜곡, 요소, 판타지, 전설]",1
9,판타지,반지의 제왕: 왕의 귀환,10,반지의 제왕 안봤다는 애들은 뭐하느라 안본건지 궁금하다,"[반지, 제왕]",1


## 5. 데이터 저장

In [33]:
import pickle

PATH = '/content/drive/MyDrive/final-project/processed_data/data/'
file_name = f'preprocessing_{genre}_0513.pkl'
with open(PATH + file_name, 'wb') as f:
    pickle.dump(df, f, protocol=4)